In [1]:
import plotly as plt
import pandas as pd
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("C:\\Users\\Tinotenda Mangarai\\Documents\\Clients\\Kudzi\\data diss\\Econet data.csv")

In [3]:
df.head()

,MSISDN,Age on Network,Recent Revenue Segment,Inactive Days,Recent Active Date,Recent Voice Usage Date,Usual Voice Appearance Week,Recent Data Usage Date,Usual Data Appearance Week,Recent SMS Usage Date,...,Recent ACS Borrow Date,Recent ACS Return Date,Churn Probability,Voice Revenue (Last 30 Days),Data Revenue (Last 30 Days),SMS Revenue (Last 30 Days),ACS Commission (Last 30 Days),Total Revenue (Last 30 Days),Recharge Amount (Last 30 Days),Region
0,263783833265,6-12 Months,HVC,29,2023-10-22,2023-10-20 00:00:00,First,2023-10-22 00:00:00,First,-,...,-,-,0.425,5633.852990,646280.015138,0.000000,0.0,651913.868128,92.490119,Harare
1,263779406999,12-24 Months,HVC,29,2023-10-22,2023-10-05 00:00:00,First,2023-10-22 00:00:00,First,-,...,-,-,0.535,2581.852193,435654.727654,0.000000,0.0,470276.625696,71.146245,Harare
2,263774021704,24+ Months,HVC,29,2023-10-22,2023-10-21 00:00:00,First,2023-10-22 00:00:00,First,2023-10-21 00:00:00,...,-,-,0.585,30604.791304,397976.284585,4090.656126,0.0,432671.732016,61.660079,Harare
3,263778829003,24+ Months,HVC,29,2023-10-22,2023-10-22 00:00:00,Second,2023-10-22 00:00:00,Second,2023-10-08 00:00:00,...,-,-,0.420,22014.850593,375841.897233,778.156522,0.0,398634.904348,57.707510,Harare
4,263789004763,12-24 Months,HVC,29,2023-10-22,2023-10-18 00:00:00,First,2023-10-22 00:00:00,First,2023-10-20 00:00:00,...,-,-,0.450,14477.571542,382401.073709,1733.543083,0.0,398612.188334,56.916996,Harare


In [4]:
# convert MSISDN to a factor 
df['MSISDN'] = df['MSISDN'].astype('category')

In [5]:
df['Recent Active Date'] = pd.to_datetime(df['Recent Active Date'])
# Convert other columns to datetime type if necessary

df['First Date of Month'] = df['Recent Active Date'] - pd.offsets.MonthBegin(1)
df['Time Difference'] = df['Recent Active Date'] - df['First Date of Month']
df['Days Since Start of Month'] = df['Time Difference'].dt.days


# Calculating VaR

In [6]:
# Calculate revenue per customer and create a new DataFrame
revenue_per_customer = df.groupby('MSISDN')['Total Revenue (Last 30 Days)'].sum()
revenue_per_customer_df = revenue_per_customer.reset_index().rename(columns={'Total Revenue (Last 30 Days)': 'Revenue per Customer'})

# Merge revenue_per_customer_df with the original DataFrame
df = df.merge(revenue_per_customer_df, on='MSISDN', how='left')

# Print the merged DataFrame
# print(df.head())

# Calculate average revenue per customer
df['Average Revenue per Customer'] = df['Revenue per Customer'].mean()

# Continue with the rest of the calculations

# Calculate the first date of the month
df['First Date of Month'] = df['Recent Active Date'] - pd.offsets.MonthBegin(1)

# Calculate the time difference
df['Time Difference'] = df['Recent Active Date'] - df['First Date of Month']

# Extract the number of days from the time difference
df['Days Since Start of Month'] = df['Time Difference'].dt.days

# Calculate CLV
df['CLV'] = df['Average Revenue per Customer'] * df['Days Since Start of Month']

# Print the DataFrame
df.head()

,MSISDN,Age on Network,Recent Revenue Segment,Inactive Days,Recent Active Date,Recent Voice Usage Date,Usual Voice Appearance Week,Recent Data Usage Date,Usual Data Appearance Week,Recent SMS Usage Date,...,ACS Commission (Last 30 Days),Total Revenue (Last 30 Days),Recharge Amount (Last 30 Days),Region,First Date of Month,Time Difference,Days Since Start of Month,Revenue per Customer,Average Revenue per Customer,CLV
0,263783833265,6-12 Months,HVC,29,2023-10-22,2023-10-20 00:00:00,First,2023-10-22 00:00:00,First,-,...,0.0,651913.868128,92.490119,Harare,2023-10-01,21 days,21,651913.868128,41953.854907,881030.953048
1,263779406999,12-24 Months,HVC,29,2023-10-22,2023-10-05 00:00:00,First,2023-10-22 00:00:00,First,-,...,0.0,470276.625696,71.146245,Harare,2023-10-01,21 days,21,470276.625696,41953.854907,881030.953048
2,263774021704,24+ Months,HVC,29,2023-10-22,2023-10-21 00:00:00,First,2023-10-22 00:00:00,First,2023-10-21 00:00:00,...,0.0,432671.732016,61.660079,Harare,2023-10-01,21 days,21,432671.732016,41953.854907,881030.953048
3,263778829003,24+ Months,HVC,29,2023-10-22,2023-10-22 00:00:00,Second,2023-10-22 00:00:00,Second,2023-10-08 00:00:00,...,0.0,398634.904348,57.707510,Harare,2023-10-01,21 days,21,398634.904348,41953.854907,881030.953048
4,263789004763,12-24 Months,HVC,29,2023-10-22,2023-10-18 00:00:00,First,2023-10-22 00:00:00,First,2023-10-20 00:00:00,...,0.0,398612.188334,56.916996,Harare,2023-10-01,21 days,21,398612.188334,41953.854907,881030.953048


In [7]:
features = [
    'Age on Network',
    'Recent Revenue Segment',
    'Inactive Days',
    'Recent Active Date',
    'Recent Voice Usage Date',
    'Usual Voice Appearance Week',
    'Recent Data Usage Date',
    'Usual Data Appearance Week',
    'Recent SMS Usage Date',
    'Usual SMS Appearance Week',
    'Recent Recharge Date',
    'Recent ACS Borrow Date',
    'Recent ACS Return Date',
    'Churn Probability',
    'Voice Revenue (Last 30 Days)',
    'Data Revenue (Last 30 Days)',
    'SMS Revenue (Last 30 Days)',
    'ACS Commission (Last 30 Days)',
    'Total Revenue (Last 30 Days)',
    'Recharge Amount (Last 30 Days)',
    'Region',
    'Days Since Start of Month',
    'Revenue per Customer',
    'CLV'
]

# Create a DataFrame with selected features
selected_df = df[features].copy()

In [8]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

k = 10  # Number of top-k features to select

# Define your feature matrix X and target variable y
X = selected_df.drop(columns=['CLV'])
y = selected_df['CLV']

# Apply label encoding to non-numeric variables
non_numeric_cols = X.select_dtypes(exclude=['number']).columns
label_encoder = LabelEncoder()

for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Perform feature normalization using Min-Max scaling
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X_normalized, y)

feature_indices = selector.get_support(indices=True)
feature_scores = selector.scores_[feature_indices]
selected_feature_names = X.columns[feature_indices]
selected_feature_names

Index(['Inactive Days', 'Recent Active Date', 'Recent Voice Usage Date',
       'Recent Data Usage Date', 'Recent SMS Usage Date',
       'Recent Recharge Date', 'Churn Probability',
       'Total Revenue (Last 30 Days)', 'Days Since Start of Month',
       'Revenue per Customer'],
      dtype='object')

In [9]:
feature_scores

array([8.53052961e+04, 8.53052961e+04, 9.65740682e+04, 8.11346490e+04,
       9.38450545e+04, 2.07982456e+05, 1.79506177e+05, 8.84897197e+04,
       1.07655307e+16, 8.84897197e+04])

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into train and test sets using the selected features
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [ ]:
# Initialize and train the SVR model
model = SVR()
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred_svr = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression evaluation metrics
mse = mean_squared_error(y_test, y_pred_svr)
mae = mean_absolute_error(y_test, y_pred_svr)
r2 = r2_score(y_test, y_pred_svr)

# Display the regression evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

# Linear Regression 

In [ ]:
features = [
    'Age on Network',
    'Recent Revenue Segment',
    'Inactive Days',
    'Recent Active Date',
    'Recent Voice Usage Date',
    'Usual Voice Appearance Week',
    'Recent Data Usage Date',
    'Usual Data Appearance Week',
    'Recent SMS Usage Date',
    'Usual SMS Appearance Week',
    'Recent Recharge Date',
    'Recent ACS Borrow Date',
    'Recent ACS Return Date',
    'Churn Probability',
    'Voice Revenue (Last 30 Days)',
    'Data Revenue (Last 30 Days)',
    'SMS Revenue (Last 30 Days)',
    'ACS Commission (Last 30 Days)',
    'Total Revenue (Last 30 Days)',
    'Recharge Amount (Last 30 Days)',
    'Region',
    'Days Since Start of Month',
    'Revenue per Customer',
    'CLV'
]

# Create a DataFrame with selected features
selected_df = df[features].copy()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

k = 10  # Number of top-k features to select

# Define your feature matrix X and target variable y
X = selected_df.drop(columns=['CLV'])
y = selected_df['CLV']

# Apply label encoding to non-numeric variables
non_numeric_cols = X.select_dtypes(exclude=['number']).columns
label_encoder = LabelEncoder()

for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Perform feature normalization using Min-Max scaling
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X_normalized, y)

feature_indices = selector.get_support(indices=True)
feature_scores = selector.scores_[feature_indices]
selected_feature_names = X.columns[feature_indices]
selected_feature_names

In [8]:
X = selected_df[['Voice Revenue (Last 30 Days)',
    'Data Revenue (Last 30 Days)',
    'SMS Revenue (Last 30 Days)',
    'ACS Commission (Last 30 Days)',
    'Total Revenue (Last 30 Days)',
    'Recharge Amount (Last 30 Days)',
    'Days Since Start of Month',
    'Revenue per Customer']]

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [13]:
# Create and train the Ridge regression model with regularization (alpha = 0.1) to prevent overfitting
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.1)
model.fit(X_train, y_train)

Ridge(alpha=0.1)

In [15]:
# Predict CLV using the trained model
y_pred = model.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate regression evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the regression evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 4.942780320972184e-08
Mean Absolute Error (MAE): 0.00012490697067602625
R-squared (R2) Score: 1.0


# A Deep ANN OR Multilayer Perceptron for Regression 

# Selecting features using SelectKBest and Mutual INFORMATION SCORES 

In [7]:
features = [
    'Age on Network',
    'Recent Revenue Segment',
    'Inactive Days',
    'Recent Active Date',
    'Recent Voice Usage Date',
    'Usual Voice Appearance Week',
    'Recent Data Usage Date',
    'Usual Data Appearance Week',
    'Recent SMS Usage Date',
    'Usual SMS Appearance Week',
    'Recent Recharge Date',
    'Recent ACS Borrow Date',
    'Recent ACS Return Date',
    'Churn Probability',
    'Voice Revenue (Last 30 Days)',
    'Data Revenue (Last 30 Days)',
    'SMS Revenue (Last 30 Days)',
    'ACS Commission (Last 30 Days)',
    'Total Revenue (Last 30 Days)',
    'Recharge Amount (Last 30 Days)',
    'Region',
    'Days Since Start of Month',
    'Revenue per Customer',
    'CLV'
]

# Create a DataFrame with selected features
selected_df = df[features].copy()

In [8]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

k = 10  # Number of top-k features to select

# Define your feature matrix X and target variable y
X = selected_df.drop(columns=['CLV'])
y = selected_df['CLV']

# Apply label encoding to non-numeric variables
non_numeric_cols = X.select_dtypes(exclude=['number']).columns
label_encoder = LabelEncoder()

for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Perform feature normalization using Min-Max scaling
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X_normalized, y)

feature_indices = selector.get_support(indices=True)
feature_scores = selector.scores_[feature_indices]
selected_feature_names = X.columns[feature_indices]
selected_feature_names

Index(['Inactive Days', 'Recent Active Date', 'Recent Voice Usage Date',
       'Recent Data Usage Date', 'Recent SMS Usage Date',
       'Recent Recharge Date', 'Churn Probability',
       'Total Revenue (Last 30 Days)', 'Days Since Start of Month',
       'Revenue per Customer'],
      dtype='object')

# Mutual Information Regression

In [11]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import mutual_info_regression

# Define your feature matrix X and target variable y
X = selected_df.drop(columns=['CLV'])
y = selected_df['CLV']

# Apply label encoding to non-numeric variables
non_numeric_cols = X.select_dtypes(exclude=['number']).columns
label_encoder = LabelEncoder()

for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Perform feature normalization using Min-Max scaling
scaler = MinMaxScaler()
X_normalised = scaler.fit_transform(X)


In [ ]:
# Calculating mutual information scores 
mi_scores = mutual_info_regression(X_normalised, y)

In [ ]:
# Convert X_normalised back to a DataFrame
X_normalized_df = pd.DataFrame(X_normalised, columns=X.columns)

# Create a DataFrame with feature names and mutual information scores
mi_df = pd.DataFrame({'Feature': X_normalized_df.columns, 'MI Score': mi_scores})

# Sort the DataFrame by MI Score in descending order
mi_df = mi_df.sort_values('MI Score', ascending=False)

# Plot the mutual information scores
plt.figure(figsize=(10, 6))
plt.barh(mi_df['Feature'], mi_df['MI Score'])
plt.xlabel('Mutual Information Score')

In [9]:
X = selected_df[['Voice Revenue (Last 30 Days)',
    'Data Revenue (Last 30 Days)',
    'SMS Revenue (Last 30 Days)',
    'ACS Commission (Last 30 Days)',
    'Total Revenue (Last 30 Days)',
    'Recharge Amount (Last 30 Days)',
    'Days Since Start of Month',
    'Revenue per Customer']]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [17]:
import tensorflow as tf
from tensorflow.keras import models, layers

# Define the model architecture
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(8,)))  # Adjust the input shape based on the number of selected features
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))  # Output layer for regression

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
163445/163445 [==============================] - 905s 6ms/step - loss: 37467754496.0000 - val_loss: 34706907136.0000
Epoch 2/10
163445/163445 [==============================] - 596s 4ms/step - loss: 3408476160.0000 - val_loss: 199935104.0000
Epoch 3/10
163445/163445 [==============================] - 827s 5ms/step - loss: 1763360640.0000 - val_loss: 4381851648.0000
Epoch 4/10
163445/163445 [==============================] - 638s 4ms/step - loss: 1253081088.0000 - val_loss: 1915306368.0000
Epoch 5/10
163445/163445 [==============================] - 595s 4ms/step - loss: 1370446080.0000 - val_loss: 78112368.0000
Epoch 6/10
163445/163445 [==============================] - 561s 3ms/step - loss: 898296192.0000 - val_loss: 115328336.0000
Epoch 7/10
163445/163445 [==============================] - 516s 3ms/step - loss: 647061248.0000 - val_loss: 2057050752.0000
Epoch 8/10
163445/163445 [==============================] - 470s 3ms/step - loss: 611179776.0000 - val_loss: 397235616.000

In [19]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test loss:", loss)

40862/40862 [==============================] - 93s 2ms/step - loss: 4411146240.0000
Test loss: 4411146240.0


In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

40862/40862 [==============================] - 102s 2ms/step
Mean Absolute Error (MAE): 42447.13751179437
Mean Squared Error (MSE): 4411163020.872257
R-squared: 0.8100504760537524


# Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from tensorflow.keras import models, layers
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd



# Define a function to create the model
def create_model(optimizer='adam', epochs=10, batch_size=32):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(8,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Define the hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [10, 20, 30],
    'batch_size': [16, 32, 64]
}

# Create a custom scoring function
def custom_scorer(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    return -r2

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Initialize lists to store results
best_score = -np.inf
best_params = {}

# Loop over the hyperparameters
for optimizer in param_grid['optimizer']:
    for epochs in param_grid['epochs']:
        for batch_size in param_grid['batch_size']:
            # Create the model
            model = create_model(optimizer=optimizer, epochs=epochs, batch_size=batch_size)

            # Perform cross-validation
            kf = KFold(n_splits=3)
            scores = []
            for train_index, val_index in kf.split(X_train):
                X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

                model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, verbose=0)
                y_pred = model.predict(X_val_fold)
                score = custom_scorer(y_val_fold, y_pred)
                scores.append(score)

            avg_score = np.mean(scores)

            # Check if this set of hyperparameters gives a better score
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'optimizer': optimizer, 'epochs': epochs, 'batch_size': batch_size}

# Print the best hyperparameters and the corresponding score
print("Best Hyperparameters: ", best_params)
print("Best Score: ", best_score)

54482/54482 [==============================] - 80s 1ms/step


In [ ]:
# Print the best hyperparameters and the corresponding score
print("Best Hyperparameters: ", best_params)
print("Best Score: ", best_score)

In [ ]:
# Save the best model
best_model.save("best_DNNmodel.h5")

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from tensorflow.keras import models, layers
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# Define a function to create the model
def create_model(optimizer='adam', epochs=10, batch_size=32):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(8,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Define the hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [5, 10, 15],
    'batch_size': [16, 32, 64]
}

# Create a custom scoring function
def custom_scorer(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    return -r2

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Initialize lists to store results
best_score = -np.inf
best_params = {}
best_model = None

# Loop over the hyperparameters
for optimizer in param_grid['optimizer']:
    for epochs in param_grid['epochs']:
        for batch_size in param_grid['batch_size']:
            # Create the model
            model = create_model(optimizer=optimizer, epochs=epochs, batch_size=batch_size)

            # Perform cross-validation
            kf = KFold(n_splits=3)
            scores = []
            for train_index, val_index in kf.split(X_train):
                X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
                y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

                model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, verbose=0)
                y_pred = model.predict(X_val_fold)
                score = custom_scorer(y_val_fold, y_pred)
                scores.append(score)

            avg_score = np.mean(scores)

            # Check if this set of hyperparameters gives a better score
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'optimizer': optimizer, 'epochs': epochs, 'batch_size': batch_size}
                best_model = model

# Print the best hyperparameters and the corresponding score
print("Best Hyperparameters: ", best_params)
print("Best Score: ", best_score)

# Save the best model
best_model.save("best_model.h5")

In [ ]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("C:\\Users\\Kudzai Nyangari\\Documents\\Project\\deep_ann_model.h5")

# Print the model summary
model.summary()